In [ ]:
from datasets import load_dataset
import torch
import pandas as pd
import torch
import random
import numpy as np
import re
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel

# Fix all seeds
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
pd.options.display.max_colwidth = None


In [2]:
socialiqa = load_dataset("allenai/social_i_qa", trust_remote_code=True)

In [ ]:

# Updated prompt including context and three answer choices
chat_prompt = """
### Instruction:
{}

### Context:
{}

### Question:
{}

### Choices:
A: {}
B: {}
C: {}

### Response:
{}"""

def load_dataset_validation(dataset_name, split):
    """Load the validation split of a dataset."""
    print(f"Loading dataset: {dataset_name} [{split}]")
    dataset = load_dataset(dataset_name, split=split)
    return dataset

def load_model(model_name, max_seq_length=2048, dtype=None, load_in_4bit=True, device="cuda"):
    """Load a model and prepare it for inference."""
    print(f"Loading model: {model_name}")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)
    model.to(device)
    return model, tokenizer

def get_letter_from_response(text):
    # Look for an isolated letter A-C (ignore D and E as SocialIQA has three options)
    match = re.search(r"\b[ABC]\b", text, re.IGNORECASE)
    if match:
        return match.group(0).upper()
    return None

def get_true_answer(label):
    # Convert the string label to int and map to a letter
    label_idx = int(label)
    mapping = {1: 'A', 2: 'B', 3: 'C'}
    return mapping.get(label_idx, None)

def evaluate_model(model, tokenizer, dataset, device="cuda", model_name="Model"):
    correct = 0
    total = len(dataset)
    invalid_responses = 0
    instruction = "Answer the multiple-choice question below based on the provided context. Be emphathetic. Your response must be a single letter: A, B, or C."
    
    for i in range(total):
        context = dataset[i]['context']
        question = dataset[i]['question']
        answerA = dataset[i]['answerA']
        answerB = dataset[i]['answerB']
        answerC = dataset[i]['answerC']
        true_answer = get_true_answer(dataset[i]['label'])

        prompt = chat_prompt.format(instruction, context, question, answerA, answerB, answerC, "")
        inputs = tokenizer([prompt], return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                use_cache=True
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "### Response:" in decoded:
            model_response = decoded.split("### Response:")[-1].strip()
        else:
            model_response = decoded.strip()
            
        predicted_letter = get_letter_from_response(model_response)

        if predicted_letter is None:
            invalid_responses += 1
        elif predicted_letter == true_answer:
            correct += 1
        # print(f"model response {model_response}")
        # print(f"predicted_letter: {predicted_letter}")
        # print(f"true_answer: {true_answer}")
    accuracy = correct / total
    print(f"[{model_name}] Accuracy: {accuracy:.2%} ({correct}/{total})")
    print(f"[{model_name}] Invalid Responses: {invalid_responses}/{total} ({invalid_responses/total:.2%})")
    return accuracy


def create_psychological_df(model, tokenizer, dataset, device="cuda", model_name="Model"):
    results = []
    
    instruction = "Answer the multiple-choice question below based on the provided context. Be emphathetic. Your response must be a single letter: A, B, or C."
    
    
    for i in range(len(dataset)):
        context = dataset[i]['context']
        question = dataset[i]['question']
        answerA = dataset[i]['answerA']
        answerB = dataset[i]['answerB']
        answerC = dataset[i]['answerC']
        
        prompt = chat_prompt.format(instruction, context, question, answerA, answerB, answerC, "")
        inputs = tokenizer([prompt], return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,  # Increased to capture reasoning
                use_cache=True
            )
        
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        
        if "### Response:" in decoded:
            model_response = decoded.split("### Response:")[-1].strip()
        else:
            model_response = decoded.strip()
            
        predicted_letter = get_letter_from_response(model_response)


        # Extract predicted letter and reasoning
        # predicted_letter = get_letter_from_response(decoded)
        reasoning = decoded if predicted_letter is None else decoded.replace(predicted_letter, "").strip()
        
        # Append results
        results.append({
            "id":i,
            "context": context,
            "question": question,
            "answer": predicted_letter
        })
    
    # Create DataFrame
    df = pd.DataFrame(results)
    return df


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
df_logic = pd.read_csv("csvs/phi_common_tuned_with_psych_q.csv")
df_psych = pd.read_csv("csvs/phi_social_psych_q.csv")

In [12]:
validation_data = load_dataset_validation("allenai/social_i_qa", "validation")
# print("validation data 1: ",validation_data)
dataset = validation_data.select(range(100))
i = 0
context = dataset[i]['context']
question = dataset[i]['question']
answerA = dataset[i]['answerA']
answerB = dataset[i]['answerB']
answerC = dataset[i]['answerC']
true_answer = get_true_answer(dataset[i]['label'])



Loading dataset: allenai/social_i_qa [validation]


In [16]:
print(context)
print(df_logic.iloc[0]["context"])
print(df_psych.iloc[0]["context"])

Tracy didn't go home that evening and resisted Riley's attacks.
Tracy didn't go home that evening and resisted Riley's attacks.
Tracy didn't go home that evening and resisted Riley's attacks.


In [ ]:
# import argparse
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
# Load test data
validation_data = load_dataset_validation("allenai/social_i_qa", "validation")
# print("validation data 1: ",validation_data)
validation_data = validation_data.select(range(100))
# print("validation data 2: ",validation_data)
device = "cuda" if torch.cuda.is_available() else "cpu"
max_seq_length = 2048
dtype = None
load_in_4bit = True


print("\nLoading fine-tuned model (LoRA)...")
fine_tuned_model_name = "phi_lora_social_qa"
# fine_tuned_model_name = "phi_lora_commonsense_qa"

finetune_model, finetune_tokenizer = FastLanguageModel.from_pretrained(
    model_name=fine_tuned_model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(finetune_model)
finetune_model.to(device)

print("\Creating psychological df ...")
psych_df = create_psychological_df(finetune_model, finetune_tokenizer, validation_data, device="cuda", model_name="Fine-Tuned Model")
########################


Loading dataset: allenai/social_i_qa [validation]

Loading fine-tuned model (LoRA)...
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Unsloth: Will load phi_lora_commonsense_qa as a legacy tokenizer.


\Creating psychological df ...


In [6]:
psych_df

,id,context,question,answer
0,0,Tracy didn't go home that evening and resisted Riley's attacks.,What does Tracy need to do before this?,C
1,1,Sydney walked past a homeless woman asking for change but did not have any money they could give to her. Sydney felt bad afterwards.,How would you describe Sydney?,B
2,2,Sasha protected the patients' rights by making new laws regarding cancer drug trials.,What will patients want to do next?,C
3,3,Jordan was in charge of taking the food on the camping trip and left all the food at home.,How would Jordan feel afterwards?,A
4,4,Kendall opened their mouth to speak and what came out shocked everyone.,How would you describe Kendall?,C
...,...,...,...,...
95,95,Riley talked to their friends about what they should do that night.,Why did Riley do this?,C
96,96,Remy went into town and found a fun place to get treats and ate some ice cream.,What does Remy need to do before this?,C
97,97,Carson was at a friend's house playing video games to get away from his arguing parents.,What does Carson need to do before this?,B
98,98,Taylor taught math in the schools and got a lot of praise for his style of teaching.,Why did Taylor do this?,A


In [7]:
psych_df.to_csv("csvs/phi_common_tuned_with_psych_q.csv")

In [ ]:
# import argparse
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset




# Load test data
validation_data = load_dataset_validation("allenai/social_i_qa", "validation")
# print("validation data 1: ",validation_data)

validation_data = validation_data.select(range(100))
# print("validation data 2: ",validation_data)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_seq_length = 2048
dtype = None
load_in_4bit = True
##################
# print("Loading original model...")
# original_model_name = "unsloth/Phi-4"
# original_model, original_tokenizer = load_model(
#     model_name=original_model_name,
#     max_seq_length=max_seq_length,
#     load_in_4bit=load_in_4bit,
#     device=device,
# )
# FastLanguageModel.for_inference(original_model)
# original_model.to(device)

# print("\nEvaluating original model...")
# evaluate_model(original_model, original_tokenizer, validation_data, device=device, model_name="Original Model")
#########################
# ----------------------------------------------------------------------
# 2) Load the FINE-TUNED model
# ----------------------------------------------------------------------

print("\nLoading fine-tuned model (LoRA)...")
fine_tuned_model_name = "phi_lora_social_qa"
# fine_tuned_model_name = "phi_lora_commonsense_qa"

finetune_model, finetune_tokenizer = FastLanguageModel.from_pretrained(
    model_name=fine_tuned_model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(finetune_model)
finetune_model.to(device)

print("\nEvaluating fine-tuned model...")
evaluate_model(finetune_model, finetune_tokenizer, validation_data, device=device, model_name="Fine-Tuned Model")
########################


Loading dataset: allenai/social_i_qa [validation]

Loading fine-tuned model (LoRA)...
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Unsloth: Will load phi_lora_commonsense_qa as a legacy tokenizer.
Unsloth 2025.2.12 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.



Evaluating fine-tuned model...
[Fine-Tuned Model] Accuracy: 71.00% (71/100)
[Fine-Tuned Model] Invalid Responses: 0/100 (0.00%)


0.71